In [2]:
import os

from sagemaker import get_execution_role
from sagemaker.huggingface import HuggingFace
from sagemaker.inputs import TrainingInput

In [3]:
# role = os.environ("SM_EXECUTION_ROLE")
role = get_execution_role()

In [4]:
# hyperparameters sent by the client are passed as command-line arguments to the script.
hyperparameters={
    "epochs": 1,
    "per_device_train_batch_size": 32,
    "model_name_or_path": "distilbert-base-cased"
}

In [5]:
# SageMaker metrics automatically parses training job logs for metrics and sends them
# to CloudWatch. If you want SageMaker to parse the logs, you must specify the metric’s
# name and a regular expression for SageMaker to use to find the metric.
metric_definitions = [
    {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
    {"Name": "eval_accuracy", "Regex": "eval_accuracy.*=\D*(.*?)$"},
    {"Name": "eval_loss", "Regex": "eval_loss.*=\D*(.*?)$"},
]

In [10]:
huggingface_estimator = HuggingFace(
    entry_point="start.py",
    source_dir="./src",
    instance_type="ml.p2.xlarge",
    instance_count=1,
    role=role,
    transformers_version="4.26.0",
    pytorch_version="1.13.1",
    py_version="py39",
    hyperparameters=hyperparameters,
)

In [11]:
data_location = "s3://sagemaker-project-p-lo6kmrzwou9t/processed/sample/distilbert-base-cased"

In [12]:
# The input dictionary is keyed on channel name.
# If using multiple channels for training data, you can specify a dict mapping channel names to strings or TrainingInput() 
# objects or FileSystemInput() objects.
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework.fit
inputs = {
    "train": TrainingInput(s3_data=f"{data_location}/train/"),
    "test": TrainingInput(s3_data=f"{data_location}/test/"),
}

In [13]:
huggingface_estimator.fit(inputs=inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-05-29-16-16-45-453


Using provided s3_resource
2023-05-29 16:16:45 Starting - Starting the training job...
2023-05-29 16:17:11 Starting - Preparing the instances for training.........
2023-05-29 16:18:26 Downloading - Downloading input data...
2023-05-29 16:18:51 Training - Downloading the training image...........................
2023-05-29 16:23:23 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-29 16:23:52,972 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-29 16:23:52,984 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-29 16:23:52,997 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-29 16:23:52,999 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-29 16:23:53,230 sagemaker-training-t

UnexpectedStatusException: Error for Training job huggingface-pytorch-training-2023-05-29-16-16-45-453: Failed. Reason: AlgorithmError: InstallRequirementsError:
ExitCode 1
ErrorMessage "ERROR: Invalid requirement: 'evaluate=0.4.0' (from line 1 of requirements.txt)
 Hint: = is not a valid operator. Did you mean == ?
 [notice] A new release of pip is available: 23.0 -> 23.1.2
 [notice] To update, run: pip install --upgrade pip"
Command "/opt/conda/bin/python3.9 -m pip install -r requirements.txt", exit code: 1